# Intro
Fitbit API implementation based loosely on https://github.com/stephenjhsu/fitbit/blob/master/fitbit.py and oauth2 tutorial at https://testdriven.io/blog/oauth-python/

This notebook covers API access, data conversion to pandas dataframes and exporting to csv.

In [17]:
# imports
import datetime
from credentials import client_id, client_secret
import pandas as pd

# import fitbit from python-fitbit folder
import sys
sys.path.append('python-fitbit')
import fitbit
import gather_keys_oauth2 as Oauth2

In [3]:
# authorize

# hoping this will work using imported creds
# client_id = '238LP7'
# client_secret = 'caea8247c1afe56879ddb2e06047966c'
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
    
# for obtaining Access-token and Refresh-token
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
 
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])

 
# Authorization
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)


[20/Jun/2022:10:43:12] ENGINE Listening for SIGTERM.
[20/Jun/2022:10:43:12] ENGINE Listening for SIGHUP.
[20/Jun/2022:10:43:12] ENGINE Listening for SIGUSR1.
[20/Jun/2022:10:43:12] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[20/Jun/2022:10:43:12] ENGINE Started monitor thread 'Autoreloader'.
[20/Jun/2022:10:43:13] ENGINE Serving on http://127.0.0.1:8080
[20/Jun/2022:10:43:13] ENGINE Bus STARTED


127.0.0.1 - - [20/Jun/2022:10:44:10] "GET /?code=4539f0747a7a3fa01f0db6883c4794e4459e8b7d&state=9uyqNRxvZJOWwbDIAgstozCdRWILqn HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"


[20/Jun/2022:10:44:11] ENGINE Bus STOPPING
[20/Jun/2022:10:44:11] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[20/Jun/2022:10:44:11] ENGINE Stopped thread 'Autoreloader'.
[20/Jun/2022:10:44:11] ENGINE Bus STOPPED
[20/Jun/2022:10:44:11] ENGINE Bus EXITING
[20/Jun/2022:10:44:11] ENGINE Bus EXITED
[20/Jun/2022:10:44:11] ENGINE Waiting for child threads to terminate...


In [4]:
#set days from first day I used fitbit to yesterday so I have all full day data
dayOne = datetime.datetime(2022, 4, 21)
yesterday = datetime.date.today() - datetime.timedelta(days=1)

In [5]:
# not needed - prints out help. Note these data are automatically available to "Personal" application type. 
# # help(auth2_client.intraday_time_series)

rawHeartData = auth2_client.intraday_time_series('activities/heart', base_date=dayOne, detail_level='1min')
rawHeartData

{'activities-heart': [{'dateTime': '2022-04-21',
   'value': {'customHeartRateZones': [],
    'heartRateZones': [{'caloriesOut': 2433.97836,
      'max': 107,
      'min': 30,
      'minutes': 1396,
      'name': 'Out of Range'},
     {'caloriesOut': 294.77645999999993,
      'max': 132,
      'min': 107,
      'minutes': 42,
      'name': 'Fat Burn'},
     {'caloriesOut': 16.89534,
      'max': 163,
      'min': 132,
      'minutes': 2,
      'name': 'Cardio'},
     {'caloriesOut': 0, 'max': 220, 'min': 163, 'minutes': 0, 'name': 'Peak'}],
    'restingHeartRate': 58}}]}

# Data extraction

## Step data

Step data is a dictionary with a list of dicts within, each consisting {dateTime: date}, {value: number_steps}, so just needs extracting, converting date to datetime, and renaming.

In [6]:
# download raw data
allStepData = auth2_client.time_series('activities/steps', base_date = dayOne, end_date = yesterday)

# look at data
allStepData

{'activities-steps': [{'dateTime': '2022-04-21', 'value': '16273'},
  {'dateTime': '2022-04-22', 'value': '9631'},
  {'dateTime': '2022-04-23', 'value': '10293'},
  {'dateTime': '2022-04-24', 'value': '9349'},
  {'dateTime': '2022-04-25', 'value': '10533'},
  {'dateTime': '2022-04-26', 'value': '13094'},
  {'dateTime': '2022-04-27', 'value': '20030'},
  {'dateTime': '2022-04-28', 'value': '10282'},
  {'dateTime': '2022-04-29', 'value': '16288'},
  {'dateTime': '2022-04-30', 'value': '17677'},
  {'dateTime': '2022-05-01', 'value': '14357'},
  {'dateTime': '2022-05-02', 'value': '14449'},
  {'dateTime': '2022-05-03', 'value': '11932'},
  {'dateTime': '2022-05-04', 'value': '15867'},
  {'dateTime': '2022-05-05', 'value': '19135'},
  {'dateTime': '2022-05-06', 'value': '20103'},
  {'dateTime': '2022-05-07', 'value': '14399'},
  {'dateTime': '2022-05-08', 'value': '13112'},
  {'dateTime': '2022-05-09', 'value': '14064'},
  {'dateTime': '2022-05-10', 'value': '21094'},
  {'dateTime': '2022-0

In [23]:
# create df
allStepDF = pd.DataFrame(allStepData.get('activities-steps'))

# rename col1
allStepDF.rename({'dateTime': 'date', 'value': 'steps'}, axis=1, inplace=True)

# convert datatype if desired allStepDF['date'] = pd.to_datetime(allStepDF['date'], format='%Y-%m-%d')

# preview data and confirm datatypes
print('Data preview: \n' 
        +str(allStepDF.head()) +'\n'
        +'\n'
        'Check datatypes: \n' 
        +str(allStepDF.dtypes))

Data preview: 
         date  steps
0  2022-04-21  16273
1  2022-04-22   9631
2  2022-04-23  10293
3  2022-04-24   9349
4  2022-04-25  10533

Check datatypes: 
date     object
steps    object
dtype: object


In [24]:
# print to csv
allStepDF.to_csv('working/allStep.csv')

## Sleep data

In [8]:
# download raw data
allSleepData = auth2_client.time_series('sleep', base_date=dayOne, end_date = yesterday)

# take a look at the data
allSleepData


{'sleep': [{'awakeCount': 5,
   'awakeDuration': 9,
   'awakeningsCount': 20,
   'dateOfSleep': '2022-06-19',
   'duration': 27240000,
   'efficiency': 93,
   'endTime': '2022-06-19T06:33:30.000',
   'logId': 37326218333,
   'minuteData': [{'dateTime': '22:59:00', 'value': '1'},
    {'dateTime': '23:00:00', 'value': '2'},
    {'dateTime': '23:01:00', 'value': '2'},
    {'dateTime': '23:02:00', 'value': '2'},
    {'dateTime': '23:03:00', 'value': '3'},
    {'dateTime': '23:04:00', 'value': '2'},
    {'dateTime': '23:05:00', 'value': '2'},
    {'dateTime': '23:06:00', 'value': '2'},
    {'dateTime': '23:07:00', 'value': '2'},
    {'dateTime': '23:08:00', 'value': '2'},
    {'dateTime': '23:09:00', 'value': '2'},
    {'dateTime': '23:10:00', 'value': '3'},
    {'dateTime': '23:11:00', 'value': '2'},
    {'dateTime': '23:12:00', 'value': '3'},
    {'dateTime': '23:13:00', 'value': '1'},
    {'dateTime': '23:14:00', 'value': '1'},
    {'dateTime': '23:15:00', 'value': '1'},
    {'dateTime':

In [9]:
# create dataframe containing data for dateOfSleep, duration, endTime, startTime

# # bypass header and access list of sleep data
dailyAllSleepData = allSleepData.get('sleep')

# note I have not extracted duration and calculated instead of imported

# create lists to store data that will go into dataframe
sleepDateList = []
wakeTimeList = []
asleepTimeList = []

# access sleep data for each day by iterating through list of length all data
for i in range(len(dailyAllSleepData)):
    # append to relevant list
    sleepDateList.append(dailyAllSleepData[i].get('dateOfSleep'))
    wakeTimeList.append(dailyAllSleepData[i].get('endTime'))
    asleepTimeList.append(dailyAllSleepData[i].get('startTime'))
    # convert list datatypes to datetime.datetime
    sleepDateList[i] = datetime.datetime.strptime(sleepDateList[i], "%Y-%m-%d")
    wakeTimeList[i] = datetime.datetime.strptime(wakeTimeList[i], "%Y-%m-%dT%H:%M:%S.%f")
    asleepTimeList[i] = datetime.datetime.strptime(asleepTimeList[i], "%Y-%m-%dT%H:%M:%S.%f")
    #generate date from alseeptime
    
    
# create and populate allSleepDF dataframe
allSleepDF = pd.DataFrame({'date': sleepDateList, 'sleepTime': asleepTimeList, 'wakeTime': wakeTimeList})

# calculate duration
allSleepDF['duration'] = allSleepDF['wakeTime'] - allSleepDF['sleepTime']


# preview data and confirm datatypes
print('Data preview: \n' 
        +str(allSleepDF.head()) +'\n'
        +'\n'
        'Check datatypes: \n' 
        +str(allSleepDF.dtypes))


Data preview: 
        date           sleepTime            wakeTime        duration
0 2022-06-19 2022-06-18 22:59:00 2022-06-19 06:33:30 0 days 07:34:30
1 2022-06-18 2022-06-17 22:57:30 2022-06-18 06:32:00 0 days 07:34:30
2 2022-06-17 2022-06-16 23:29:00 2022-06-17 06:34:30 0 days 07:05:30
3 2022-06-16 2022-06-15 23:12:00 2022-06-16 06:33:30 0 days 07:21:30
4 2022-06-15 2022-06-14 23:34:00 2022-06-15 06:38:00 0 days 07:04:00

Check datatypes: 
date          datetime64[ns]
sleepTime     datetime64[ns]
wakeTime      datetime64[ns]
duration     timedelta64[ns]
dtype: object


In [18]:
# print to csv
allSleepDF.to_csv('allSleep.csv')

## Heart Rate Data

In [10]:
# download raw data
allHeartData = auth2_client.time_series('activities/heart', base_date = dayOne, end_date = yesterday)


In [11]:
# download raw data
allHeartData = auth2_client.time_series('activities/heart', base_date = dayOne, end_date = yesterday)

# take a look at data
allHeartData

{'activities-heart': [{'dateTime': '2022-04-21',
   'value': {'customHeartRateZones': [],
    'heartRateZones': [{'caloriesOut': 2433.97836,
      'max': 107,
      'min': 30,
      'minutes': 1396,
      'name': 'Out of Range'},
     {'caloriesOut': 294.77645999999993,
      'max': 132,
      'min': 107,
      'minutes': 42,
      'name': 'Fat Burn'},
     {'caloriesOut': 16.89534,
      'max': 163,
      'min': 132,
      'minutes': 2,
      'name': 'Cardio'},
     {'caloriesOut': 0, 'max': 220, 'min': 163, 'minutes': 0, 'name': 'Peak'}],
    'restingHeartRate': 58}},
  {'dateTime': '2022-04-22',
   'value': {'customHeartRateZones': [],
    'heartRateZones': [{'caloriesOut': 2382.6627000000003,
      'max': 107,
      'min': 30,
      'minutes': 1437,
      'name': 'Out of Range'},
     {'caloriesOut': 20.14848,
      'max': 132,
      'min': 107,
      'minutes': 3,
      'name': 'Fat Burn'},
     {'caloriesOut': 0,
      'max': 163,
      'min': 132,
      'minutes': 0,
      'name

Here, I see a dictionary with key 'activities-heart', and a separate dictionary for each day. Each day contains a list of dictionaries, for example:
{'activities-heart': [{'dateTime': '2022-04-21',
   'value': {'customHeartRateZones': [],
    'heartRateZones': [{'caloriesOut': 2433.97836,
      'max': 107,
      'min': 30,
      'minutes': 1396,
      'name': 'Out of Range'},
     {'caloriesOut': 294.77645999999993,
      'max': 132,
      'min': 107,
      'minutes': 42,
      'name': 'Fat Burn'},
     {'caloriesOut': 16.89534,
      'max': 163,
      'min': 132,
      'minutes': 2,
      'name': 'Cardio'},
     {'caloriesOut': 0, 'max': 220, 'min': 163, 'minutes': 0, 'name': 'Peak'}],
    'restingHeartRate': 58}}
    
  Breaking this down, we have 3x key:value pairs:

dateTime = date
value = HR zone data

Within value there are 4 heart rate zones: ('Out of Range', 'Fat Burn', 'Cardio', 'restingHeartRate').
The 3 zones each contain:
max - maximum observed bpm in this zone
min - minimum observed bpm in this zone
minutes - number of minutes in this zone
name - name of the zone

and restingHeartRate = calculated resting heart rate (measured during sleep)

{'dateTime': '2022-04-21',
   'value': {'customHeartRateZones': [], 'heartRateZones': 
                                          [{'caloriesOut': 2433.97836,
      'max': 107,
      'min': 30,
      'minutes': 1396,
      'name': 'Out of Range'},
     {'caloriesOut': 294.77645999999993,
      'max': 132,
      'min': 107,
      'minutes': 42,
      'name': 'Fat Burn'},
     {'caloriesOut': 16.89534,
      'max': 163,
      'min': 132,
      'minutes': 2,
      'name': 'Cardio'},
     {'caloriesOut': 0, 'max': 220, 'min': 163, 'minutes': 0, 'name': 'Peak'}],
    'restingHeartRate': 58}}


    So, to extract, I'll need to access:
    1. dateTime
    2. values
    3. restingHeartRate

    1. dateTime needs extracting and converting to datetime object
    2. values
      Here I'll need to access the nested dict and extract max, min, minutes and name
    3. restingHeartRate just need extracting and storing as int

    My end goal is a dataframe with separate columns for each dataset, ie:  [date, Out of Range_max, Out of Range_min, Out of Range_minutes, Fat Burn_max, Fat Burn_min, Fat Burn_minutes, Cardio_max, Cardio_min, Cardio_minutes, restingHeartRate]. Note I'm not exporting calories as I'm not interested in them.

In [12]:
# create dataframe containing data for dateOfSleep, duration, endTime, startTime

# # bypass header and access list of sleep data
# note I have not extracted duration and calculated instead of imported



# create lists to store data that will go into dataframe
sleepDateList = []
wakeTimeList = []
asleepTimeList = []

# access sleep data for each day by iterating through list of length all data
for i in range(len(dailyAllSleepData)):
    # append to relevant list
    sleepDateList.append(dailyAllSleepData[i].get('dateOfSleep'))
    wakeTimeList.append(dailyAllSleepData[i].get('endTime'))
    asleepTimeList.append(dailyAllSleepData[i].get('startTime'))
    # convert list datatypes to datetime.datetime
    sleepDateList[i] = datetime.datetime.strptime(sleepDateList[i], "%Y-%m-%d")
    wakeTimeList[i] = datetime.datetime.strptime(wakeTimeList[i], "%Y-%m-%dT%H:%M:%S.%f")
    asleepTimeList[i] = datetime.datetime.strptime(asleepTimeList[i], "%Y-%m-%dT%H:%M:%S.%f")
    #generate date from alseeptime
    
    
# create and populate allSleepDF dataframe
allSleepDF = pd.DataFrame({'date': sleepDateList, 'sleepTime': asleepTimeList, 'wakeTime': wakeTimeList})

# calculate duration
allSleepDF['duration'] = allSleepDF['wakeTime'] - allSleepDF['sleepTime']


# preview data and confirm datatypes
print('Data preview: \n' 
        +str(allSleepDF.head()) +'\n'
        +'\n'
        'Check datatypes: \n' 
        +str(allSleepDF.dtypes))


Data preview: 
        date           sleepTime            wakeTime        duration
0 2022-06-19 2022-06-18 22:59:00 2022-06-19 06:33:30 0 days 07:34:30
1 2022-06-18 2022-06-17 22:57:30 2022-06-18 06:32:00 0 days 07:34:30
2 2022-06-17 2022-06-16 23:29:00 2022-06-17 06:34:30 0 days 07:05:30
3 2022-06-16 2022-06-15 23:12:00 2022-06-16 06:33:30 0 days 07:21:30
4 2022-06-15 2022-06-14 23:34:00 2022-06-15 06:38:00 0 days 07:04:00

Check datatypes: 
date          datetime64[ns]
sleepTime     datetime64[ns]
wakeTime      datetime64[ns]
duration     timedelta64[ns]
dtype: object


In [13]:
# bypass header and get daily data dicts
dailyAllHeartData = allHeartData.get('activities-heart')
dailyAllHeartData

[{'dateTime': '2022-04-21',
  'value': {'customHeartRateZones': [],
   'heartRateZones': [{'caloriesOut': 2433.97836,
     'max': 107,
     'min': 30,
     'minutes': 1396,
     'name': 'Out of Range'},
    {'caloriesOut': 294.77645999999993,
     'max': 132,
     'min': 107,
     'minutes': 42,
     'name': 'Fat Burn'},
    {'caloriesOut': 16.89534,
     'max': 163,
     'min': 132,
     'minutes': 2,
     'name': 'Cardio'},
    {'caloriesOut': 0, 'max': 220, 'min': 163, 'minutes': 0, 'name': 'Peak'}],
   'restingHeartRate': 58}},
 {'dateTime': '2022-04-22',
  'value': {'customHeartRateZones': [],
   'heartRateZones': [{'caloriesOut': 2382.6627000000003,
     'max': 107,
     'min': 30,
     'minutes': 1437,
     'name': 'Out of Range'},
    {'caloriesOut': 20.14848,
     'max': 132,
     'min': 107,
     'minutes': 3,
     'name': 'Fat Burn'},
    {'caloriesOut': 0, 'max': 163, 'min': 132, 'minutes': 0, 'name': 'Cardio'},
    {'caloriesOut': 0, 'max': 220, 'min': 163, 'minutes': 0, '

In [14]:
# create lists to store data that will go into df
heartDateList = []
outOfRangeList = []
fatBurnList = []
cardioList = []
restingHeartRateList = []

for i in range(len(dailyAllHeartData)):
    # append date
    heartDateList.append(dailyAllHeartData[i].get('dateTime'))
    # create daily heart rate value object to save chaining .get commands for the rest
    dailyZoneValues = dailyAllHeartData[i].get('value').get('heartRateZones')
    # here, dailyZoneValues[0] = Out of range, dailyZoneValues[1] = fat burn, dailyZoneValues[2] = cardio. I'm not looping to simplify adding to different lists
    outOfRangeList.append([dailyZoneValues[0].get('max'), dailyZoneValues[0].get('min'), dailyZoneValues[0].get('minutes')])
    fatBurnList.append([dailyZoneValues[1].get('max'), dailyZoneValues[1].get('min'), dailyZoneValues[1].get('minutes')])
    cardioList.append([dailyZoneValues[2].get('max'), dailyZoneValues[2].get('min'), dailyZoneValues[2].get('minutes')])
    # append resting heart rate
    restingHeartRateList.append(dailyAllHeartData[0].get('value').get('restingHeartRate'))
    # convert dates to datetime
    heartDateList[i] = datetime.datetime.strptime(heartDateList[i], "%Y-%m-%d")

# create dataframe for heartDateList and restingHeartList
heartDateAndRestingDF = pd.DataFrame({'date': heartDateList, 'restingHeartRate': restingHeartRateList})

# outOfRanteList, fatBurnList, cardiList are lists of tuples, so unpack into dataframes
outOfRangeDF = pd.DataFrame(outOfRangeList, columns=['normal_max', 'normal_min', 'normal_minutes']) # note renamed 'normal' as opposed to 'Out of Range'
fatBurnDF = pd.DataFrame(fatBurnList, columns=['fatBurn_max', 'fatBurn_min', 'fatBurn_minutes'])
cardioDF = pd.DataFrame(cardioList, columns=['cardio_max', 'cardio_min', 'cardio_minutes'])

# merge
allHeartDF = pd.concat([heartDateAndRestingDF, outOfRangeDF, fatBurnDF, cardioDF], axis=1)

# preview data and confirm datatypes
print('Data preview: \n' 
        +str(allHeartDF.head()) +'\n'
        +'\n'
        'Check datatypes: \n' 
        +str(allHeartDF.dtypes))

Data preview: 
        date  restingHeartRate  normal_min  normal_max  normal_minutes  \
0 2022-04-21                58         107          30            1396   
1 2022-04-22                58         107          30            1437   
2 2022-04-23                58         107          30            1405   
3 2022-04-24                58         106          30            1439   
4 2022-04-25                58         107          30            1411   

   fatBurn_min  fatBurn_max  fatBurn_minutes  cardio_min  cardio_max  \
0          132          107               42         163         132   
1          132          107                3         163         132   
2          132          107               35         163         132   
3          131          106                1         163         131   
4          132          107               28         163         132   

   cardio_minutes  
0               2  
1               0  
2               0  
3               0  
4      

In [21]:
# print to csv
allHeartDF.to_csv('working/allHeart.csv')